In [1]:
import numpy as np
from audio_batch import batch_audio, get_valid
from reg_nn import reg_nn

In [2]:
sin_pre = np.load('./data/lowpass/pre/sin.npz')['data']
sin_post = np.load('./data/lowpass/post/sin.npz')['data']

In [3]:
print('audio shapes should match:', sin_pre.shape, sin_post.shape)

audio shapes should match: (2, 1228496) (2, 1228496)


In [4]:
set_in = np.concatenate([sin_pre[0], sin_pre[1]])
set_out = np.concatenate([sin_post[0], sin_post[1]])

In [5]:
train_in, train_out, valid_in, valid_out = get_valid(set_in, set_out, 1, .25)

In [6]:
train_in.shape[0] + valid_in.shape[0] == train_out.shape[0] + valid_out.shape[0] == set_in.shape[0] == set_out.shape[0]

True

In [23]:
input_set, output_set = batch_audio(train_in, train_out, .5, offset=.1)

In [24]:
valid_in_batches  = valid_in.reshape(int(valid_in.shape[0] / input_set.shape[1]), input_set.shape[1])
valid_out_batches = valid_out.reshape(int(valid_out.shape[0] / output_set.shape[1]), output_set.shape[1])

In [25]:
sess, optimizer, x, y, P, MSE = reg_nn(input_set.shape[1], output_set.shape[1], 8, 1024)

In [ ]:
epochs = 3

for i in range(epochs):
    perm = np.random.permutation(input_set.shape[0])

    for batch in np.arange(len(perm)):
        x_batch = input_set[perm[batch]]
        y_batch = output_set[perm[batch]]
        (_, mse_train, p_train) = sess.run([optimizer, MSE, P],feed_dict={x:[x_batch],y:[y_batch]})
        (mse_valid, p_valid)    = sess.run([MSE,P],feed_dict={x:valid_in_batches,y:valid_out_batches})
        if (batch % 10 == 0):
            print(i, batch, mse_train, (y_batch - p_train).std(), mse_valid, (valid_out_batches - p_valid).std())
        
print(batch, mse_train, (y_batch.mean() - p_train.mean()).std(), mse_valid, (valid_out_batches.mean() - p_valid.mean()).std())

0 0 0.00709292 0.0842141566275 0.0163611 0.127909
0 10 0.0072039 0.0848705163151 0.0163462 0.127851
0 20 0.018109 0.134567146744 0.0163311 0.127792
0 30 0.0108993 0.104369715716 0.016316 0.127733
0 40 0.00859603 0.0927099127283 0.0163008 0.127673
0 50 0.0070088 0.0837132616303 0.0162854 0.127613
0 60 0.0169531 0.130202853178 0.0162699 0.127552
0 70 0.00697839 0.0835314841403 0.0162543 0.127491
0 80 0.0173016 0.131507883079 0.0162386 0.127429
0 90 0.00759421 0.0871397384692 0.0162228 0.127367
0 100 0.00955776 0.0977144828172 0.0162069 0.127305
0 110 0.0102516 0.101245939442 0.0161909 0.127242
0 120 0.0175726 0.132560136725 0.0161748 0.127179
0 130 0.0173114 0.131570658076 0.0161587 0.127115
0 140 0.0177504 0.133224744773 0.0161425 0.127051
0 150 0.0081741 0.0904059300778 0.0161262 0.126987
0 160 0.0370781 0.192554861396 0.0161098 0.126923
0 170 0.00682095 0.0825837687198 0.0160933 0.126858
0 180 0.026165 0.161753298741 0.0160766 0.126792
0 190 0.00844575 0.0918960793521 0.0160598 0.1267